# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import six
from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
from student import *
import simple_mdp as sm

In [3]:
n_concepts = 4
use_student2 = True
transition_after = True
student2_str = ('2' if use_student2 else '') + ('a' if use_student2 and transition_after else '')
learn_prob = 0.5
lp_str = '-lp{}'.format(int(learn_prob*100)) if not use_student2 else ''
n_students = 100000
seqlen = 5
filter_mastery = False
filter_str = '' if not filter_mastery else '-filtered'
policy = 'random'
epsilon = 0.3
epsilon_str = '{:.2f}'.format(epsilon) if policy == 'egreedy' else ''
filename = 'test{}-n{}-l{}{}-{}{}{}.pickle'.format(student2_str, n_students, seqlen,
                                                    lp_str, policy, epsilon_str, filter_str)
#concept_tree = sm.create_custom_dependency()
concept_tree = ConceptDependencyGraph()
concept_tree.init_default_tree(n_concepts)
if not use_student2:
    test_student = Student(n=n_concepts,p_trans_satisfied=learn_prob, p_trans_not_satisfied=0.0, p_get_ex_correct_if_concepts_learned=1.0)
else:
    test_student = Student2(n_concepts, transition_after=transition_after)
print(filename)

test2a-n100000-l5-random.pickle


In [4]:
# Generates the data
# Only run this cell if need to generate new data, otherwise skip this
if False:
    print ("Initializing synthetic data sets...")
    dg.generate_data(concept_tree, student=test_student, n_students=n_students, filter_mastery=filter_mastery, seqlen=seqlen, policy=policy, epsilon=epsilon, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
    print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 5.
Data generation completed. 


In [5]:
import dynamics_model_class as dmc
import mcts_tests as mc
import mcts
import numpy as np
import dataset_utils
import tensorflow as tf
import tflearn
import copy

In [6]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))

In [7]:
print('Average posttest: {}'.format(sm.expected_reward(data)))
print('Average sparse reward: {}'.format(sm.expected_sparse_reward(data)))
print('Percent of full posttest score: {}'.format(sm.percent_complete(data)))
print('Percent of all seen: {}'.format(sm.percent_all_seen(data)))
six.print_(data[0])
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

Average posttest: 0.4377175
Average sparse reward: 0.0
Percent of full posttest score: 0.0
Percent of all seen: 0.38222
[(array([ 0.,  0.,  1.,  0.]), False, array([ 1.,  0.,  0.,  0.])), (array([ 0.,  0.,  0.,  1.]), False, array([ 1.,  0.,  0.,  0.])), (array([ 0.,  1.,  0.,  0.]), False, array([ 1.,  0.,  0.,  0.])), (array([ 0.,  0.,  1.,  0.]), False, array([ 1.,  0.,  1.,  0.])), (array([ 1.,  0.,  0.,  0.]), True, array([ 1.,  0.,  1.,  0.]))]


In [8]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])
print(input_data_.shape)
print(output_mask_.shape)
print(target_data_.shape)
six.print_(input_data_[0,:,:])
six.print_(output_mask_[0,:,:])
six.print_(target_data_[0,:,:])

(100000, 5, 8)
(100000, 5, 4)
(100000, 5, 4)
[[ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [10]:
# test_model hidden=16
# test_model_mid hidden=10
# test_model_small hidden=5
# test_model_tiny hidden=3
model_id = "test2_model_small"
dmodel = dmc.DynamicsModel(model_id=model_id, timesteps=seqlen, dropout=1.0, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test2_model_small/
Checkpoint directory path: ../checkpoints/test2_model_small/
Model loaded.


In [11]:
class ExtractCallback(tflearn.callbacks.Callback):
    def __init__(self):
        self.tstate = None
    def on_epoch_end(self, training_state):
        self.tstate = copy.copy(training_state)
ecall = ExtractCallback()

In [12]:
dmodel.train(train_data, n_epoch=10, callbacks=ecall, load_checkpoint=False)

Training Step: 14069  | total loss: 0.00088 | time: 6.547s
| Adam | epoch: 010 | loss: 0.00088 -- iter: 89984/90000
Training Step: 14070  | total loss: 0.00087 | time: 7.714s
| Adam | epoch: 010 | loss: 0.00087 | val_loss: 0.00085 -- iter: 90000/90000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [13]:
print(dir(ecall.tstate))
print(ecall.tstate.global_loss)
print(ecall.tstate.val_loss)

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', 'acc_value', 'best_accuracy', 'current_iter', 'epoch', 'global_acc', 'global_loss', 'increaseEpoch', 'increaseStep', 'loss_value', 'resetGlobal', 'step', 'step_time', 'step_time_total', 'update', 'val_acc', 'val_loss']
0.000871833821293
0.000853506263532


In [14]:
# save the model to a checkpoint file
chkpt = 'tempmodel'
dmodel.save(chkpt)

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [15]:
# test the model on the real environment
test_horizon = 6
n_rollouts = 1000
n_trajectories = 100
r_type = mcts.SPARSE

test_student = Student2(n_concepts, transition_after=transition_after)
test_student.reset()
test_student.knowledge[0] = 1 # initialize the first concept to be known
sim = StudentExactSim(test_student.copy(), concept_tree)

mc.test_dkt_chunk(n_trajectories, concept_tree, sim, model_id, chkpt, test_horizon, n_rollouts, False, r_type)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test2_model_small/
Checkpoint directory path: ../checkpoints/test2_model_small/
Model loaded.
INFO:tensorflow:Restoring parameters from /usr0/home/zguo/Documents/smart-tutor/code/tempmodel


(100.0, 93.971984540507989)